In [1]:
from taxcalc import *
import numpy as np
import pandas as pd
from pandas import DataFrame
from IPython.display import display

In [2]:
# settings
# Change year here if running simulations for any other year
sim_year = 2017
agi_brackets = [(-1e14, 50000), (50000, 100000), (100000, 1e14)]
# list of dictionaries for each elasticity
elasticities = [
    {'lower': -0.5, 'middle': -0.5, 'upper': -0.5},
    {'lower': -1.0, 'middle': -1.0, 'upper': -1.0},
    {'lower': -2.236, 'middle': -1.490, 'upper': -1.182}]

In [3]:
# lists for each output
base_list = list()
std_list = list()
charitable_list = list()
lower_mtr_list = list()
std_mtr_list = list()
std_charitable_list = list()
all_three_list = list()

In [4]:
# Reform dictionaries

# 2: raise standard deduction
std_reform = {
    sim_year: {
        '_STD': [[11000, 22000, 11000, 11000, 22000]]
    }
}

# 3: allow non-itemizers charitable deduction
charitable_reform = {
    sim_year: {
        '_STD_allow_charity_ded_nonitemizers': [True]
    }
}

# 4: lower top MTR to 35%
mtr_reform = {
    sim_year: {
        '_II_rt7': [0.35]
    }
}

In [5]:
# average charitable contributions by AGI level
contribs = pd.Series([342.96, 857.90, 1585.56])
recs = Records()
pol = Policy()

# blow-up contributions to 2013 levels
for year in range(2010, 2014):
    contribs *= recs.gfactors.factor_value('ATXPY', year)
contribs

0     389.598564
1     974.564405
2    1801.177687
dtype: float64

In [6]:
# intitalize calculator - used to assign charitable giving to non-itemizers
calc = Calculator(records=recs, policy=pol)
calc.calc_all()

You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.


In [7]:
# assign contribution amounts to non-itemizers based on AGI
calc.records.e19800 = np.where((calc.records.c00100 < 50000) & (calc.records.c04470 == 0),
                               contribs[0], calc.records.e19800)
calc.records.e19800 = np.where((calc.records.c00100 >= 50000) & (calc.records.c00100 < 100000) &
                               (calc.records.c04470 == 0),
                               contribs[1], calc.records.e19800)
calc.records.e19800 = np.where((calc.records.c00100 > 100000) & (calc.records.c04470 == 0),
                               contribs[2], calc.records.e19800)

In [8]:
# generate output table
# Total charitable giving only consists of cash contributions. Non-cash can be easily added

def table(calc_table):
    
    dictionary = {'Revenue': [],
                  'Total Giving': [], 
                  'Total Giving: Itemizers': [], 
                  'Total Giving: Non-Itemizers': [], 
                  'Number of Itemizers': [], 
                  'Number of Non-Itemizers': []}
    # Loop through each AGI income bin to get values
    for item in agi_brackets:
        
        val = calc_table.records._combined[(calc_table.records.c00100 >= item[0])&
                                           (calc_table.records.c00100 < item[1])]
        val *= calc_table.records.s006[(calc_table.records.c00100 >= item[0])&
                                       (calc_table.records.c00100 < item[1])]
        dictionary['Revenue'].append(val.sum() / 1000000)
        
        val = calc_table.records.e19800[(calc_table.records.c00100 >= item[0])&
                                        (calc_table.records.c00100 < item[1])]
        val *= calc_table.records.s006[(calc_table.records.c00100 >= item[0])&
                                       (calc_table.records.c00100 < item[1])]
        dictionary['Total Giving'].append(float(val.sum()) / 1000000)
        
        val = calc_table.records.e19800[(calc_table.records.c00100 >= item[0])&
                                        (calc_table.records.c00100 < item[1])&
                                        (calc_table.records.c04470 > 0)]
        val *= calc_table.records.s006[(calc_table.records.c00100 >= item[0])&
                                       (calc_table.records.c00100 < item[1])&
                                       (calc_table.records.c04470 > 0)]
        dictionary['Total Giving: Itemizers'].append(float(val.sum()) / 1000000)
        
        val = calc_table.records.s006[(calc_table.records.c00100 >= item[0])&
                                      (calc_table.records.c00100 < item[1])&
                                      (calc_table.records.c04470 > 0)]
        dictionary['Number of Itemizers'].append(float(val.sum()) / 1000000)

        val = calc_table.records.e19800[(calc_table.records.c00100 >= item[0])&
                                        (calc_table.records.c00100 < item[1])&
                                        (calc_table.records.c04470 == 0)]
        val *= calc_table.records.s006[(calc_table.records.c00100 >= item[0])&
                                       (calc_table.records.c00100 < item[1])&
                                       (calc_table.records.c04470 == 0)]
        dictionary['Total Giving: Non-Itemizers'].append(float(val.sum()) / 1000000)
        
        val = calc_table.records.s006[(calc_table.records.c00100 >= item[0])&
                                      (calc_table.records.c00100 < item[1])&
                                      (calc_table.records.c04470 == 0)]
        dictionary['Number of Non-Itemizers'].append(float(val.sum()) / 1000000)
        
    df = DataFrame.from_dict(dictionary)
    df.index = ['$0-50K', '$50K-100K', '$100K+']
    df = df.round(2)
        
    return(df)


# Begin Simulations

In [9]:
# calculator with no reforms or elasticities to be used as base
calc_noref = Calculator(records=Records(), policy=Policy())
calc_noref.advance_to_year(sim_year)

You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.


In [10]:
# baseline function - no policy changes, but with elasticities
def base(elast):
    """
    elast: dictionary of elasticities
    returns DF to make the table we're interested in
    """
    # Set calculator
    recs_base = Records()
    pol_base = Policy()
    beh_base = Behavior()
    calc_base = Calculator(records=recs_base, policy=pol_base, behavior=beh_base,
                          verbose=False)
    beh_base.update_behavior({
        sim_year: {
            '_BE_charity': [[elast['lower'], elast['middle'], elast['upper']]]
        }
    })
    calc_base.records.e19800 = calc.records.e19800
    
    # assuming we're doing the analysis for 2017
    calc_base.advance_to_year(sim_year)
    
    # create calculator with behavioral response
    calc_base_beh = Behavior.response(calc_noref, calc_base)
    
    # Create and append DF with information interested in
    base_list.append(table(calc_base_beh))

In [11]:
# increase value of the standard deduction ($11,000 for individuals and $22,000 for joint filers,
# following the Camp proposal)
def std(elast):
    recs_std = Records()
    pol_std = Policy()
    pol_std.implement_reform(std_reform)
    beh_std = Behavior()
    calc_std = Calculator(records=recs_std, policy=pol_std, behavior=beh_std,
                          verbose=False)
    beh_std.update_behavior({
        sim_year: {
            '_BE_charity': [[elast['lower'], elast['middle'], elast['upper']]]
        }
    })
    calc_std.records.e19800 = calc.records.e19800
    calc_std.advance_to_year(sim_year)
    
    # Create calculator with behavioral reform
    calc_std_beh = Behavior.response(calc_noref, calc_std)
    
    # create and append DF with information interested in
    std_list.append(table(calc_std_beh))

In [12]:
# ability for non-itemizers to itemize charitable deductions 
def charitable(elast):
    recs_charitable = Records()
    pol_charitable = Policy()
    pol_charitable.implement_reform(charitable_reform)
    beh_charitable = Behavior()
    calc_charitable = Calculator(records=recs_charitable, policy=pol_charitable, behavior=beh_charitable,
                                 verbose=False)
    beh_charitable.update_behavior({
        sim_year: {
            '_BE_charity': [[elast['lower'], elast['middle'], elast['upper']]]
        }
    })
    calc_charitable.records.e19800 = calc.records.e19800
    calc_charitable.advance_to_year(sim_year)
    
    # Create calculator with behaviroal reform
    calc_charitable_beh = Behavior.response(calc_noref, calc_charitable)
    
    # Create and append DF with information interested in
    charitable_list.append(table(calc_charitable_beh))

In [13]:
# decrease of the highest marginal tax rate to 35% 
def lower_mtr(elast):
    recs_mtr = Records()
    pol_mtr = Policy()
    pol_mtr.implement_reform(mtr_reform)
    beh_mtr = Behavior()
    calc_mtr = Calculator(records=recs_mtr, policy = pol_mtr, behavior=beh_mtr,
                          verbose=False)
    beh_mtr.update_behavior({
        sim_year: {
            '_BE_charity': [[elast['lower'], elast['middle'], elast['upper']]]
        }
    })
    calc_mtr.records.e19800 = calc.records.e19800
    calc_mtr.advance_to_year(sim_year)
    
    # Calculator with behavioral reform
    calc_mtr_beh = Behavior.response(calc_noref, calc_mtr)
    
    # Create and append DF with information interested
    lower_mtr_list.append(table(calc_mtr_beh))

In [14]:
# standard deduction change and the marginal rate decrease
def std_mtr(elast):
    recs_std_mtr = Records()
    pol_std_mtr = Policy()
    pol_std_mtr.implement_reform(std_reform)
    pol_std_mtr.implement_reform(mtr_reform)
    beh_std_mtr = Behavior()
    calc_std_mtr = Calculator(records=recs_std_mtr, policy=pol_std_mtr, behavior=beh_std_mtr,
                              verbose=False)
    calc_std_mtr.records.e19800 = calc.records.e19800
    beh_std_mtr.update_behavior({
        sim_year: {
            '_BE_charity': [[elast['lower'], elast['middle'], elast['upper']]]
        }
    })
    calc_std_mtr.advance_to_year(sim_year)
    
    # Calculator with behavioral reform
    calc_std_mtr_beh = Behavior.response(calc_noref, calc_std_mtr)
    
    # Create and append df with information interested in
    std_mtr_list.append(table(calc_std_mtr_beh)) 

In [15]:
# standard deduction change AND the non-itemizer deduction change
def std_charitable(elast):
    recs_std_charitable = Records()
    pol_std_charitable = Policy()
    pol_std_charitable.implement_reform(std_reform)
    pol_std_charitable.implement_reform(charitable_reform)
    beh_std_charitable = Behavior()
    calc_std_charitable = Calculator(records=recs_std_charitable, policy=pol_std_charitable,
                                     behavior=beh_std_charitable, verbose=False)
    calc_std_charitable.records.e19800 = calc.records.e19800
    beh_std_charitable.update_behavior({
        sim_year: {
            '_BE_charity': [[elast['lower'], elast['middle'], elast['upper']]]
        }
    })
    calc_std_charitable.advance_to_year(sim_year)
    
    # Calculator with behavioral response
    calc_std_charitable_beh = Behavior.response(calc_noref, calc_std_charitable)
    
    # Create and append DF with information
    std_charitable_list.append(table(calc_std_charitable_beh))

In [16]:
# all three changes (standard deduction increase, non-itemizer deductions, and marginal tax rate decrease)
def all_three(elast):
    recs_three = Records()
    pol_three = Policy()
    pol_three.implement_reform(std_reform)
    pol_three.implement_reform(charitable_reform)
    pol_three.implement_reform(mtr_reform)
    beh_three = Behavior()
    calc_three = Calculator(records=recs_three, policy=pol_three, behavior=beh_three,
                            verbose=False)
    calc_three.records.e19800 = calc.records.e19800
    beh_three.update_behavior({
        sim_year: {
            '_BE_charity': [[elast['lower'], elast['middle'], elast['upper']]]
        }
    })
    calc_three.advance_to_year(sim_year)
    
    # Calculator with behavior reform
    calc_three_beh = Behavior.response(calc_noref, calc_three)
    
    # Create and append DF with information
    all_three_list.append(table(calc_three_beh))

In [17]:
# loop through each branch of elasticites, calling each function
for item in elasticities:
    base(item)
    std(item)
    charitable(item)
    lower_mtr(item)
    std_mtr(item)
    std_charitable(item)
    all_three(item)

taxcalc/behavior.py:306: RuntimeWarning: invalid value encountered in divide
  delta_income * calc.records.e00200 / agi_m_ided,
taxcalc/behavior.py:310: RuntimeWarning: invalid value encountered in divide
  delta_income * other_income / agi_m_ided,
taxcalc/behavior.py:313: RuntimeWarning: invalid value encountered in divide
  delta_income * ided / agi_m_ided,


# Output (In Millions)

Each table is in the form of a DataFrame and is ready to be exported if need be

# baseline function

In [39]:
# baseline function
# elasticity: {'lower': -0.5, 'middle': -0.5, 'upper': -0.5}
display(base_list[0])

,Number of Itemizers,Number of Non-Itemizers,Revenue,Total Giving,Total Giving: Itemizers,Total Giving: Non-Itemizers
$0-50K,7.45,100.64,206209.89,55614.84,10649.65,44965.19
$50K-100K,13.99,18.41,477848.85,48923.16,31852.92,17070.24
$100K+,23.85,6.08,2108964.96,140270.40,129947.25,10323.15


In [19]:
# baseline function
# elasticity: {'lower': -1.0, 'middle': -1.0, 'upper': -1.0}
display(base_list[1])

,Number of Itemizers,Number of Non-Itemizers,Revenue,Total Giving,Total Giving: Itemizers,Total Giving: Non-Itemizers
$0-50K,7.85,100.24,204888.89,140595.49,27562.02,113033.47
$50K-100K,14.37,18.03,470292.85,123732.04,81943.14,41788.90
$100K+,24.06,5.87,2055995.20,354622.34,329603.52,25018.81


In [40]:
# baseline function
# elasticity: {'lower': -2.236, 'middle': -1.490, 'upper': -1.182}
display(base_list[2])

,Number of Itemizers,Number of Non-Itemizers,Revenue,Total Giving,Total Giving: Itemizers,Total Giving: Non-Itemizers
$0-50K,8.15,99.94,203303.37,353179.66,68390.61,284789.06
$50K-100K,17.98,14.42,457965.68,306399.38,226190.28,80209.10
$100K+,27.25,2.68,1954969.38,867366.57,843994.49,23372.07


# increase value of the standard deduction

In [41]:
# increase value of the standard deduction ($11,000 for individuals and $22,000 for joint filers,
# following the Camp proposal)
# elasticity: {'lower': -0.5, 'middle': -0.5, 'upper': -0.5}
display(std_list[0])

,Number of Itemizers,Number of Non-Itemizers,Revenue,Total Giving,Total Giving: Itemizers,Total Giving: Non-Itemizers
$0-50K,3.68,104.41,184630.41,63228.27,7062.05,56166.22
$50K-100K,8.33,24.07,452488.67,54918.92,25756.44,29162.48
$100K+,18.00,11.93,2086328.71,158215.23,136472.35,21742.88


In [42]:
# increase value of the standard deduction ($11,000 for individuals and $22,000 for joint filers,
# following the Camp proposal)
# elasticity: {'lower': -1.0, 'middle': -1.0, 'upper': -1.0}
display(std_list[1])

,Number of Itemizers,Number of Non-Itemizers,Revenue,Total Giving,Total Giving: Itemizers,Total Giving: Non-Itemizers
$0-50K,4.60,103.49,183598.12,160509.58,25263.11,135246.47
$50K-100K,10.01,22.39,445480.56,141257.57,86168.83,55088.74
$100K+,19.78,10.15,2029303.07,404851.39,368093.63,36757.76


In [43]:
# increase value of the standard deduction ($11,000 for individuals and $22,000 for joint filers,
# following the Camp proposal)
# elasticity: {'lower': -2.236, 'middle': -1.490, 'upper': -1.182}
display(std_list[2])

,Number of Itemizers,Number of Non-Itemizers,Revenue,Total Giving,Total Giving: Itemizers,Total Giving: Non-Itemizers
$0-50K,5.19,102.90,182315.57,403326.95,65675.80,337651.15
$50K-100K,11.15,21.25,434763.60,349988.05,221904.60,128083.45
$100K+,21.81,8.12,1923863.66,991207.62,922548.68,68658.94


# ability for non-itemizers to itemize charitable deductions 

In [44]:
# ability for non-itemizers to itemize charitable deductions 
# elasticity: {'lower': -0.5, 'middle': -0.5, 'upper': -0.5}
display(charitable_list[0])

,Number of Itemizers,Number of Non-Itemizers,Revenue,Total Giving,Total Giving: Itemizers,Total Giving: Non-Itemizers
$0-50K,6.17,101.92,202654.84,72444.21,6724.37,65719.85
$50K-100K,11.87,20.53,471194.43,63684.08,25138.02,38546.06
$100K+,22.36,7.57,2093164.92,182022.35,150374.36,31647.99


In [45]:
# ability for non-itemizers to itemize charitable deductions
# elasticity: {'lower': -1.0, 'middle': -1.0, 'upper': -1.0}
display(charitable_list[1])

,Number of Itemizers,Number of Non-Itemizers,Revenue,Total Giving,Total Giving: Itemizers,Total Giving: Non-Itemizers
$0-50K,5.91,102.18,196969.89,183244.31,14767.47,168476.84
$50K-100K,11.95,20.45,456110.21,161265.43,66525.13,94740.30
$100K+,22.88,7.05,2023055.45,462194.94,406708.84,55486.09


In [46]:
# ability for non-itemizers to itemize charitable deductions
# elasticity: {'lower': -2.236, 'middle': -1.490, 'upper': -1.182}
display(charitable_list[2])

,Number of Itemizers,Number of Non-Itemizers,Revenue,Total Giving,Total Giving: Itemizers,Total Giving: Non-Itemizers
$0-50K,5.50,102.59,185774.85,460865.31,30609.17,430256.14
$50K-100K,12.07,20.33,429127.89,400497.02,170398.17,230098.85
$100K+,23.62,6.31,1899458.03,1133977.58,1030397.46,103580.12


# decrease of the highest marginal tax rate to 35% # Lower MTR Results

In [47]:
# decrease of the highest marginal tax rate to 35% 
# elasticity: {'lower': -0.5, 'middle': -0.5, 'upper': -0.5}
display(lower_mtr_list[0])

,Number of Itemizers,Number of Non-Itemizers,Revenue,Total Giving,Total Giving: Itemizers,Total Giving: Non-Itemizers
$0-50K,7.55,100.54,205716.29,82676.78,15935.73,66741.06
$50K-100K,14.18,18.22,475277.99,72634.14,47779.40,24854.74
$100K+,23.95,5.98,2067276.72,206871.08,191885.70,14985.38


In [48]:
# decrease of the highest marginal tax rate to 35% 
# elasticity: {'lower': -1.0, 'middle': -1.0, 'upper': -1.0}
display(lower_mtr_list[1])

,Number of Itemizers,Number of Non-Itemizers,Revenue,Total Giving,Total Giving: Itemizers,Total Giving: Non-Itemizers
$0-50K,7.97,100.12,204184.95,209199.21,41208.10,167991.11
$50K-100K,14.47,17.93,465434.96,184107.22,122280.12,61827.11
$100K+,24.39,5.54,1995644.58,527660.68,492924.90,34735.78


In [49]:
# decrease of the highest marginal tax rate to 35% 
# elasticity: {'lower': -2.236, 'middle': -1.490, 'upper': -1.182}
display(lower_mtr_list[2])

,Number of Itemizers,Number of Non-Itemizers,Revenue,Total Giving,Total Giving: Itemizers,Total Giving: Non-Itemizers
$0-50K,8.24,99.85,202639.50,526413.02,102975.73,423437.28
$50K-100K,20.10,12.30,448909.77,457838.95,360757.54,97081.42
$100K+,28.13,1.80,1872765.36,1296044.39,1276739.84,19304.55


# standard deduction change and the marginal rate decrease

In [50]:
# standard deduction change and the marginal rate decrease
# elasticity: {'lower': -0.5, 'middle': -0.5, 'upper': -0.5}
display(std_mtr_list[0])

,Number of Itemizers,Number of Non-Itemizers,Revenue,Total Giving,Total Giving: Itemizers,Total Giving: Non-Itemizers
$0-50K,3.98,104.11,184280.60,94218.99,12563.29,81655.70
$50K-100K,9.15,23.25,450260.39,82412.16,45860.77,36551.40
$100K+,18.83,11.10,2043614.95,235155.51,209259.87,25895.64


In [51]:
# standard deduction change and the marginal rate decrease
# elasticity: {'lower': -1.0, 'middle': -1.0, 'upper': -1.0}
display(std_mtr_list[1])

,Number of Itemizers,Number of Non-Itemizers,Revenue,Total Giving,Total Giving: Itemizers,Total Giving: Non-Itemizers
$0-50K,4.87,103.22,183013.59,238830.39,39014.48,199815.92
$50K-100K,10.54,21.86,440907.35,210184.35,132173.01,78011.34
$100K+,20.43,9.50,1966863.12,602399.05,552530.59,49868.46


In [52]:
# standard deduction change and the marginal rate decrease
# elasticity: {'lower': -2.236, 'middle': -1.490, 'upper': -1.182}
display(std_mtr_list[2])

,Number of Itemizers,Number of Non-Itemizers,Revenue,Total Giving,Total Giving: Itemizers,Total Giving: Non-Itemizers
$0-50K,5.50,102.58,181788.88,601151.85,100284.08,500867.77
$50K-100K,16.72,15.68,428721.36,523236.93,398112.74,125124.20
$100K+,25.06,4.87,1844179.20,1482314.02,1441122.26,41191.76


# standard deduction change AND the non-itemizer deduction change

In [53]:
# standard deduction change AND the non-itemizer deduction change
# elasticity: {'lower': -0.5, 'middle': -0.5, 'upper': -0.5}
display(std_charitable_list[0])

,Number of Itemizers,Number of Non-Itemizers,Revenue,Total Giving,Total Giving: Itemizers,Total Giving: Non-Itemizers
$0-50K,2.75,105.34,180083.91,107627.14,3775.73,103851.41
$50K-100K,6.47,25.93,439566.50,94306.54,21378.94,72927.60
$100K+,16.34,13.59,2049491.58,268903.48,207915.48,60988.00


In [54]:
# standard deduction change AND the non-itemizer deduction change
# elasticity: {'lower': -1.0, 'middle': -1.0, 'upper': -1.0}
display(std_charitable_list[1])

,Number of Itemizers,Number of Non-Itemizers,Revenue,Total Giving,Total Giving: Itemizers,Total Giving: Non-Itemizers
$0-50K,2.60,105.49,174271.74,272658.56,7809.63,264848.93
$50K-100K,6.76,25.64,420880.77,239955.07,66661.83,173293.24
$100K+,17.94,11.99,1959089.40,687723.44,591344.21,96379.23


In [55]:
# standard deduction change AND the non-itemizer deduction change
# elasticity: {'lower': -2.236, 'middle': -1.490, 'upper': -1.182}
display(std_charitable_list[2])

,Number of Itemizers,Number of Non-Itemizers,Revenue,Total Giving,Total Giving: Itemizers,Total Giving: Non-Itemizers
$0-50K,2.47,105.62,163285.93,686640.34,17233.39,669406.95
$50K-100K,7.12,25.28,389903.90,598275.95,180852.16,417423.78
$100K+,20.09,9.84,1822406.39,1696219.63,1535961.76,160257.88


# all three changes (standard deduction increase, non-itemizer deductions, and marginal tax rate decrease)

In [56]:
# all three changes (standard deduction increase, non-itemizer deductions, and marginal tax rate decrease)
# elasticity: {'lower': -0.5, 'middle': -0.5, 'upper': -0.5}
display(all_three_list[0])

,Number of Itemizers,Number of Non-Itemizers,Revenue,Total Giving,Total Giving: Itemizers,Total Giving: Non-Itemizers
$0-50K,2.73,105.35,179474.96,122868.96,4168.99,118699.96
$50K-100K,6.50,25.90,437565.29,107649.19,25108.42,82540.77
$100K+,16.50,13.43,2016655.47,306300.20,241008.05,65292.15


In [57]:
# all three changes (standard deduction increase, non-itemizer deductions, and marginal tax rate decrease)
# elasticity: {'lower': -1.0, 'middle': -1.0, 'upper': -1.0}
display(all_three_list[1])

,Number of Itemizers,Number of Non-Itemizers,Revenue,Total Giving,Total Giving: Itemizers,Total Giving: Non-Itemizers
$0-50K,2.59,105.50,173093.73,311278.19,8815.38,302462.81
$50K-100K,6.81,25.59,417323.49,273942.54,77645.38,196297.16
$100K+,18.19,11.74,1920323.31,785133.26,682675.35,102457.91


In [58]:
# all three changes (standard deduction increase, non-itemizer deductions, and marginal tax rate decrease)
# elasticity: {'lower': -2.236, 'middle': -1.490, 'upper': -1.182}
display(all_three_list[2])

,Number of Itemizers,Number of Non-Itemizers,Revenue,Total Giving,Total Giving: Itemizers,Total Giving: Non-Itemizers
$0-50K,2.46,105.63,161122.49,784217.46,19521.13,764696.33
$50K-100K,7.23,25.17,384081.92,683985.01,209512.01,474473.00
$100K+,20.71,9.22,1780679.04,1939830.13,1775687.52,164142.61
